# World War Z - A Mathmatical Model

In this course we have studied many interesting and pratical numerical soultion problems intended to give us a better understanding of some of the typical engineering problems we as engineers could encounter in our career.  For my final project I have choosen a topic that while interesting to model, is likely never going to be encountered.  With that said lets start a model for the zombie appocalyse.

## Background

The first step in modeling for this dooms day scenario would be to basically define what a zombie is.  Per wikipedia, zombies are fictional undead creatures created through the reanimation of human corpses.  Usually in these doomsday senerious some virus is inadventurently introduced into the general population with an initial patient, commonly refered to as "patient zero" being the first zombie.  Zombies don't require anything like food or water to live however they do have a insasibable appiteite for human flesh.  The zombie virus is then spread through the population by bites from infected indiviuals.  My project will work under the assumption that this outbreak has already occured however there is a "safe" community inside of a walled city that is keeping all the infected at bay.

So what's there to model right?  All the undead are outside of the city with only the living inside.  Well lets take a page out of the AMC tv show The Walking Dead and assume that everyone in the world is already infected with this zombie virus and it doesn't matter if you die from a zombie bite or natural causes, you still have the potential to come back as a zombie as long as the cause of death didn't come from a serve brain injury (which is of course the only way to kill a zombie).  For this model we will assume that someone, somewhere inside our city died from natural causes and came back as our patient zero.

## SZR Model 

A good starting place to find some useful equations would be to look at the classic SIR model.  This model is commonly used to model the effect of a rapid infection on a group of people in contact with one another.  Sounds like as good of a place as any to start doesn't it?

The SIR model works by divided the population into one of three groups.    

    -  S: "Susceptibles" - This is the number of current healthy individuals that could potentially become infected.
    -  I: "Infectious" - This is the number of individuals infected with the virus.
    -  R: "Recovered" - This is the number of individuals who have recovered from the virus.  

In order to predict the effect of the virus over the population over time a few other variables need to be defined as well.

    - Beta "The contact rate" - This is the probability that a susceptible would become infected if the two were to meet.
    - "" "The rate of recovery" - This is expressed as 1/(duration of infection), i.e., in reciprocal units of time.
    
The following differential equations are then obtained to express the behavior of the infection over time.

    {dS}{dt} = beta I S 

    {dI}{dt} = beta I S - nu I

    {dR}{dt} = nu I

Thats great for modeling something like the flu, but we are talking about zombies here!  Fortunitely for us some professors up in Ottawa came up with a modified version of this model to be used for zombies.  The whole paper can be found (insert link here) 

The SZR model has three compartments, as before, but the transitions between them are more complex.

    - β – the probability that a susceptible becomes a zombie when the two encounter each other.

    - α – the rate at which zombies are slain, by removing their heads or destroying their brains.

    - ζ – the rate at which humans in the removed class resurrect to become zombies.
    
    - Π - Birth rate
    
    - δ - The rate at which the susceptibles die from something other then a zombie bite

This then gives us the following differential equations.

    {dS}{dt} = Π − βSZ − δS

    {dZ}{dt} = βSZ + ζR − αSZ

    {dR}{dt} = δS + αSZ − ζR
    
Since we are only modeling a short time period here we can assume that the birth rate Π is 0.  Additionally we don't expect there to be any deaths that wouldn't result in reanimation as even deaths from something other then a zombie bite would still produce a zombie.  Therefore we can assume δ is 0 as well.  Finally we assume that humans who are bite do not go into the removed group but instead are converted straight to a zombie.  This simplifies the equations giving us the following differential equations to use for our model.

    {dS}{dt} = −βSZ

    {dZ}{dt} = βSZ − αSZ = (β−α)SZ

    {dR}{dt} = αSZ

## Reaction-Diffusion 

diffusion of the zombie population into the susceptible population we need to define how they will react to each other when they collide.  For this model we will assume only two possible outcomes.

(1)  The susceptible kills the zombie by damaging its brain.

or

(2)  The zombie bites the susceptible converting him to a zombie who is now free to attack and infect other susceptibles.

These outcomes can easily be written as a chemical reaction similar to what we had seen in the fourth HW assignment.

S + Z ----> S (at rate alpha)
S + Z ----> Z + Z (at rate beta)

According to the Law of Mass Action, the rate of any chemical reaction is proportional to the product of the masses of the reacting substances


In [367]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [368]:
Ds = .00008
Dz = .00016

n = 192
dh = 5/(n-1)
T = 8000
dt = .9 * dh**2 / (4*max(Ds,Dz))
nt = int(T/dt)

beta = .25                  # The probability that a susceptible becomes a zombie when the two encounter each other.
alpha = .75                 # The rate at which zombies are killed

In [369]:
def ftcs(S, Z, nt, dt, dh):
   
    for n in range(nt):
        Sn = S.copy()
        Zn = Z.copy()
        
        S[1:-1,1:-1] = Sn[1:-1,1:-1] + Ds *(dt/dh**2 * (Sn[2:,1:-1] - 2*Sn[1:-1,1:-1] + Sn[:-2,1:-1]) + dt/dh**2 * (Sn[1:-1,2:] - 2*Sn[1:-1,1:-1] + Sn[1:-1,:-2])) - (beta * Sn[1:-1,1:-1] * Zn[1:-1,1:-1] * dt)
        Z[1:-1,1:-1] = Zn[1:-1,1:-1] + Dz *(dt/dh**2 * (Zn[2:,1:-1] - 2*Zn[1:-1,1:-1] + Zn[:-2,1:-1]) + dt/dh**2 * (Zn[1:-1,2:] - 2*Zn[1:-1,1:-1] + Zn[1:-1,:-2])) + ((beta-alpha) * Sn[1:-1,1:-1] * Zn[1:-1,1:-1] * dt)   
        
        # Neumann conditions everywhere
        S[0,:] = S[1,:]
        S[:,0] = S[:,1]
        S[-1,:] = S[-2,:]
        S[:,-1] = S[:,-2]
        Z[0,:] = Z[1,:]
        Z[:,0] = Z[:,1]
        Z[-1,:] = Z[-2,:]
        Z[:,-1] = Z[:,-2]
        
    return S, Z

In [370]:
S = numpy.ones((n,n))
Z = numpy.zeros_like(S)

In [371]:
Z[1,1]=1
print(Z)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [372]:
zombie = ftcs(S, Z, nt, dt, dh)

In [373]:
print(S[191])

[ 1.44297339  1.44297339  0.94947626  1.31681973  0.82731245  1.27667368
  0.82432996  1.27585902  0.82484086  1.27606324  0.82487442  1.27610988
  0.82488075  1.27615589  0.82489021  1.27620989  0.82490119  1.27627105
  0.8249135   1.27633906  0.82492708  1.27641365  0.82494187  1.27649451
  0.82495783  1.27658129  0.82497488  1.27667365  0.82499295  1.2767712
  0.82501196  1.27687354  0.82503183  1.27698026  0.82505249  1.27709091
  0.82507385  1.27720506  0.82509583  1.27732225  0.82511833  1.27744202
  0.82514126  1.27756388  0.82516454  1.27768737  0.82518807  1.27781199
  0.82521177  1.27793729  0.82523554  1.27806276  0.82525928  1.27818795
  0.82528292  1.27831238  0.82530637  1.27843559  0.82532953  1.27855713
  0.82535233  1.27867657  0.82537468  1.27879347  0.8253965   1.27890743
  0.82541773  1.27901805  0.82543828  1.27912496  0.82545808  1.2792278
  0.82547708  1.27932622  0.82549522  1.27941992  0.82551242  1.27950859
  0.82552865  1.27959198  0.82554386  1.27966982  0.8

In [374]:
print(Z[191])

[ 0.37663505  0.37663505 -1.40187251  0.84770308 -1.67566206  0.82477634
 -1.6745554   0.82096265 -1.67318017  0.82099317 -1.67324352  0.82104926
 -1.67336403  0.82109515 -1.67349841  0.82114787 -1.67365183  0.82120762
 -1.67382389  0.82127408 -1.67401386  0.82134696 -1.67422095  0.82142597
 -1.6744443   0.82151076 -1.67468299  0.821601   -1.67493604  0.82169632
 -1.67520241  0.82179631 -1.67548101  0.82190057 -1.67577071  0.82200869
 -1.67607035  0.82212022 -1.67637873  0.82223471 -1.67669461  0.82235172
 -1.67701674  0.82247077 -1.67734386  0.82259141 -1.6776747   0.82271317
 -1.67800797  0.82283557 -1.67834241  0.82295815 -1.67867673  0.82308044
 -1.67900968  0.82320199 -1.67934003  0.82332235 -1.67966656  0.82344108
 -1.67998808  0.82355775 -1.68030343  0.82367195 -1.6806115   0.82378326
 -1.68091121  0.82389132 -1.68120153  0.82399574 -1.68148148  0.82409619
 -1.68175012  0.82419232 -1.68200658  0.82428384 -1.68225005  0.82437046
 -1.68247976  0.8244519  -1.68269503  0.82452793 -1